# Getting Started with pybrms

This notebook demonstrates how to use **pybrms** - a Pythonic interface to the brms R package.

**pybrms v0.1.0** features:
- CmdStanPy backend (official Stan interface)
- Explicit brms version control
- Support for multiple inference methods (MCMC, VI, optimization)
- Python 3.8+ compatibility

## Setup

This notebook works in two scenarios:
1. **Installed package**: `pip install pybrms`
2. **Cloned repository**: Running from repository root

In [1]:
# Import pybrms - works for both installed package and cloned repo
import sys
import os

try:
    # Try importing as installed package
    import pybrms
    print(f"✓ Using installed pybrms {pybrms.__version__}")
except ImportError:
    # Fall back to importing from cloned repo
    # Add parent directory to path (assumes running from examples/)
    repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    
    try:
        import pybrms
        print(f"✓ Using pybrms from repository: {repo_root}")
        print(f"  Version: {pybrms.__version__}")
    except ImportError as e:
        print("✗ Could not import pybrms!")
        print("  Please either:")
        print("  1. Install: pip install pybrms")
        print("  2. Or run from repository root with: jupyter notebook examples/getting_started.ipynb")
        raise

# Import other required packages
import pandas as pd
import numpy as np

# Visualization (optional)
try:
    import arviz as az
    import matplotlib.pyplot as plt
    HAS_VIZ = True
    print("✓ Visualization libraries available (arviz, matplotlib)")
except ImportError:
    HAS_VIZ = False
    print("ℹ Visualization libraries not available (install with: pip install pybrms[viz])")

print("\n" + "="*60)
print("Setup complete!")
print("="*60)

Error importing in API mode: ImportError("dlopen(/Users/sebastian/PycharmProjects/pybrms/.venv/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <05EE76C0-33BF-3D4C-BC56-84005241BC88> /Users/sebastian/PycharmProjects/pybrms/.venv/lib/python3.12/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.


✓ Using pybrms from repository: /Users/sebastian/PycharmProjects/pybrms
  Version: 0.1.0
✓ Visualization libraries available (arviz, matplotlib)

Setup complete!


## 1. First Time Setup (One-Time Only)

Install brms R package and CmdStan compiler. This only needs to be done once per environment.

In [2]:
# Check if brms and CmdStan are already installed
try:
    version = pybrms.get_brms_version()
    print(f"✓ brms already installed (version {version})")
    
    import cmdstanpy
    cmdstan_path = cmdstanpy.cmdstan_path()
    print(f"✓ CmdStan already installed at: {cmdstan_path}")
    print("\nSkipping installation - already set up!")
    
except Exception as e:
    print("Installing brms and CmdStan...")
    print("This may take a few minutes on first run.")
    print("")
    
    # Install both brms and CmdStan
    pybrms.install_brms()
    
    print("\n" + "="*60)
    print("Installation complete!")
    print("="*60)

✓ brms already installed (version 1.2.22)
✓ CmdStan already installed at: /Users/sebastian/.cmdstan/cmdstan-2.37.0

Skipping installation - already set up!


## 2. Load Example Data

pybrms provides access to all datasets included in the brms package.

In [3]:
# Load the epilepsy dataset
epilepsy = pybrms.get_brms_data("epilepsy")

print("Epilepsy Dataset")
print("="*60)
print(f"Shape: {epilepsy.shape}")
print(f"\nColumns: {', '.join(epilepsy.columns)}")
print(f"\nFirst few rows:")
epilepsy.head()

Epilepsy Dataset
Shape: (236, 9)

Columns: Age, Base, Trt, patient, visit, count, obs, zAge, zBase

First few rows:


,Age,Base,Trt,patient,visit,count,obs,zAge,zBase
0,31.0,11.0,0,1,1,5.0,1,0.424995,-0.757173
1,30.0,11.0,0,2,1,3.0,2,0.265284,-0.757173
2,25.0,6.0,0,3,1,2.0,3,-0.533274,-0.944403
3,36.0,8.0,0,4,1,4.0,4,1.223553,-0.869511
4,22.0,66.0,0,5,1,7.0,5,-1.012408,1.302363


In [4]:
# Quick data exploration
print("Data Summary")
print("="*60)
print(epilepsy.describe())

print("\nTarget Variable Distribution (count):")
print(epilepsy['count'].value_counts().sort_index().head(10))

Data Summary
              Age        Base       count          zAge         zBase
count  236.000000  236.000000  236.000000  2.360000e+02  2.360000e+02
mean    28.338983   31.220339    8.254237  1.768830e-16 -3.387121e-17
std      6.261290   26.705051   12.346891  1.000000e+00  1.000000e+00
min     18.000000    6.000000    0.000000 -1.651255e+00 -9.444033e-01
25%     23.000000   12.000000    2.750000 -8.526970e-01 -7.197267e-01
50%     28.000000   22.000000    4.000000 -5.413949e-02 -3.452657e-01
75%     32.000000   41.000000    9.000000  5.847065e-01  3.662102e-01
max     42.000000  151.000000  102.000000  2.181822e+00  4.485281e+00

Target Variable Distribution (count):
count
0.0    23
1.0    16
2.0    20
3.0    33
4.0    29
5.0    17
6.0    14
7.0    13
8.0    11
9.0     5
Name: count, dtype: int64


## 3. Fit a Simple Model

Let's fit a Poisson regression model with random effects.

**Model:** `count ~ zAge + zBase * Trt + (1|patient)`

This model includes:
- Fixed effects: `zAge`, `zBase`, `Trt` and their interaction
- Random effect: varying intercepts by patient

In [ ]:
# Fit the model
# Using fewer iterations for faster demonstration
# For real analysis, use default values (iter_warmup=1000, iter_sampling=1000)

print("Fitting Bayesian Poisson regression model...")
print("This will take a minute or two.\n")

model = pybrms.fit(
    formula="count ~ zAge + zBase * Trt + (1|patient)",
    data=epilepsy,
    family="poisson",
    warmup=250,      # Warmup iterations (for demo; use 1000+ for real analysis)
    iter=500,    # Sampling iterations (for demo; use 1000+ for real analysis) 
    chains=2       # Number of MCMC chains
)

print("\n" + "="*60)
print("Model fitting complete!")
print("="*60)

Fitting Bayesian Poisson regression model...
This will take a minute or two.

Fitting model with brms (backend: cmdstanr)...


R callback write-console: Model executable is up to date!
  
R callback write-console: Start sampling
  


Running MCMC with 2 sequential chains...

Chain 1 Iteration:   1 / 500 [  0%]  (Warmup) 
Chain 1 Iteration: 100 / 500 [ 20%]  (Warmup) 
Chain 1 Iteration: 200 / 500 [ 40%]  (Warmup) 
Chain 1 Iteration: 251 / 500 [ 50%]  (Sampling) 
Chain 1 Iteration: 350 / 500 [ 70%]  (Sampling) 
Chain 1 Iteration: 450 / 500 [ 90%]  (Sampling) 
Chain 1 Iteration: 500 / 500 [100%]  (Sampling) 
Chain 1 finished in 0.7 seconds.
Chain 2 Iteration:   1 / 500 [  0%]  (Warmup) 
Chain 2 Iteration: 100 / 500 [ 20%]  (Warmup) 
Chain 2 Iteration: 200 / 500 [ 40%]  (Warmup) 
Chain 2 Iteration: 251 / 500 [ 50%]  (Sampling) 
Chain 2 Iteration: 350 / 500 [ 70%]  (Sampling) 
Chain 2 Iteration: 450 / 500 [ 90%]  (Sampling) 


## 4. Examine Results

View the posterior summary statistics.

In [ ]:
# Get summary statistics
summary = model.summary()

print("Posterior Summary")
print("="*60)
print(summary)

# Check convergence diagnostics
print("\nConvergence Check")
print("="*60)
if 'R_hat' in summary.columns:
    max_rhat = summary['R_hat'].max()
    print(f"Max R-hat: {max_rhat:.4f}")
    if max_rhat < 1.01:
        print("✓ Excellent convergence (R-hat < 1.01)")
    elif max_rhat < 1.05:
        print("Average convergence (R-hat < 1.05)")
    else:
        print("⚠ Poor convergence (R-hat >= 1.05) - consider more iterations")
else:
    print("R-hat not available in summary")

Posterior Summary
                   Mean      MCSE    StdDev       MAD          5%         50%  \
lp__        -701.705000  0.553700  7.242070  7.186230 -714.142000 -701.459000   
b[1]           0.092171  0.006333  0.087729  0.087492   -0.053649    0.096979   
b[2]           0.720349  0.009358  0.123976  0.112004    0.511494    0.719784   
b[3]          -0.269735  0.012345  0.174504  0.167826   -0.566195   -0.268703   
b[4]           0.027466  0.011397  0.164571  0.161125   -0.247566    0.032838   
...                 ...       ...       ...       ...         ...         ...   
r_1_1[57]     -0.190789  0.010178  0.279471  0.283875   -0.673262   -0.181327   
r_1_1[58]      0.580909  0.020001  0.263339  0.235360    0.131937    0.579701   
r_1_1[59]      0.001814  0.012560  0.248994  0.249378   -0.420673    0.015996   
lprior        -3.183890  0.000700  0.009622  0.008509   -3.201770   -3.182650   
b_Intercept    1.778710  0.007814  0.120473  0.124286    1.581560    1.780110   

         

## 5. Visualization (Optional)

Visualize posterior distributions using arviz.

Inference data with groups:
	> posterior
	> sample_stats

In [ ]:
if HAS_VIZ:
    # Convert to arviz InferenceData
    idata = az.from_cmdstanpy(model)
    
    # Plot posterior distributions for fixed effects
    fig = az.plot_posterior(
        idata,
        var_names=['b_Intercept', 'b_zAge', 'b_zBase', 'b_Trt', 'b_zBase:Trt'],
        figsize=(12, 8),
        textsize=10
    )
    plt.suptitle('Posterior Distributions - Fixed Effects', y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✓ Posterior distributions plotted")
else:
    print("⚠ Visualization skipped (arviz not installed)")
    print("  Install with: pip install pybrms[viz]")

KeyError: 'var names: "[\'b_zAge\' \'b_zBase\' \'b_Trt\' \'b_zBase:Trt\'] are not present" in dataset'

In [ ]:
if HAS_VIZ:
    # Trace plots to check MCMC behavior
    az.plot_trace(
        idata,
        var_names=['b_Intercept', 'b_zAge', 'b_zBase'],
        figsize=(12, 6)
    )
    plt.suptitle('MCMC Trace Plots', y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✓ Trace plots show MCMC chain behavior")

## 6. Extract and Analyze Samples

Access the raw posterior samples for custom analysis.

In [ ]:
# Get parameter draws
draws = model.draws()

print("Posterior Samples")
print("="*60)
print(f"Shape: {draws.shape}")
print(f"  (chains, iterations, parameters)")
print(f"\nTotal posterior samples: {draws.shape[0] * draws.shape[1]}")

# Example: Extract specific parameter
if HAS_VIZ:
    # Get samples for zAge coefficient
    b_zAge_samples = idata.posterior['b_zAge'].values.flatten()
    
    print(f"\nb_zAge (effect of age):")
    print(f"  Mean: {b_zAge_samples.mean():.4f}")
    print(f"  Median: {np.median(b_zAge_samples):.4f}")
    print(f"  95% CI: [{np.percentile(b_zAge_samples, 2.5):.4f}, {np.percentile(b_zAge_samples, 97.5):.4f}]")
    
    # Simple histogram
    plt.figure(figsize=(10, 4))
    plt.hist(b_zAge_samples, bins=50, density=True, alpha=0.7, edgecolor='black')
    plt.axvline(b_zAge_samples.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    plt.xlabel('b_zAge')
    plt.ylabel('Density')
    plt.title('Posterior Distribution of zAge Coefficient')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 7. Model with Custom Priors

Specify informative priors for better inference.

In [ ]:
# Fit model with custom priors
print("Fitting model with custom priors...\n")

model_with_priors = pybrms.fit(
    formula="count ~ zAge + zBase",
    data=epilepsy,
    family="poisson",
    priors=[
        ("normal(0, 0.5)", "b"),         # Regularizing prior on coefficients
        ("normal(1, 0.5)", "Intercept")  # Informative prior on intercept
    ],
    iter_warmup=500,
    iter_sampling=500,
    chains=2,
    show_console=False
)

print("\n" + "="*60)
print("Model with priors fitted!")
print("="*60)

# Compare summaries
print("\nSummary:")
print(model_with_priors.summary())

## 8. Other Example Datasets

pybrms includes access to all brms datasets.

In [ ]:
# Load kidney dataset
kidney = pybrms.get_brms_data("kidney")
print("Kidney Dataset")
print("="*60)
print(f"Shape: {kidney.shape}")
print(f"\nFirst few rows:")
print(kidney.head())

print("\n" + "="*60)

# Load inhaler dataset
inhaler = pybrms.get_brms_data("inhaler")
print("\nInhaler Dataset")
print("="*60)
print(f"Shape: {inhaler.shape}")
print(f"\nFirst few rows:")
print(inhaler.head())

## Summary

This notebook demonstrated:
1. ✅ Setting up pybrms (installing brms and CmdStan)
2. ✅ Loading example datasets
3. ✅ Fitting Bayesian regression models
4. ✅ Examining posterior summaries and diagnostics
5. ✅ Visualizing results with arviz
6. ✅ Extracting and analyzing posterior samples
7. ✅ Specifying custom priors

### Next Steps

- Read the [MIGRATION.md](../MIGRATION.md) guide for API changes from v0.0.3
- Check [ARCHITECTURE.md](../ARCHITECTURE.md) for system design details
- See brms documentation: https://paul-buerkner.github.io/brms/
- Explore CmdStanPy features: https://mc-stan.org/cmdstanpy/

### Support

- GitHub: https://github.com/kaitumisuuringute-keskus/pybrms
- Issues: https://github.com/kaitumisuuringute-keskus/pybrms/issues

Happy modeling! 🎉